In [66]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import csv
import matplotlib.pyplot as plt
import pymongo

In [67]:
listing_host_csv = "resources/NYCAribnb.csv"
listing_host_df = pd.read_csv(listing_host_csv,encoding="utf8")

In [68]:
listing_host_df.rename(index=str,columns={"listing_name":"name"},inplace=True)
listing_host_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,10/19/2018,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,5/21/2019,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,7/5/2019,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,11/19/2018,0.10,1,0


In [69]:
# Create new data with select columns for the AirBnB listings
listing_host_df = listing_host_df[["id","name","host_id","host_name","neighbourhood_group","neighbourhood","latitude","longitude","room_type","price","minimum_nights","number_of_reviews","last_review","reviews_per_month","calculated_host_listings_count","availability_365"]].copy()

In [70]:
# convert dates to datetime
listing_host_df["last_review"]=pd.to_datetime(listing_host_df["last_review"])
listing_host_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaT,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [71]:
dict([(v, listing_host_df[v].apply(lambda r: len(str(r)) if r!=None else 0).max())for v in listing_host_df.columns.values])

{'id': 8,
 'name': 179,
 'host_id': 9,
 'host_name': 35,
 'neighbourhood_group': 13,
 'neighbourhood': 26,
 'latitude': 18,
 'longitude': 18,
 'room_type': 15,
 'price': 5,
 'minimum_nights': 4,
 'number_of_reviews': 3,
 'last_review': 19,
 'reviews_per_month': 5,
 'calculated_host_listings_count': 3,
 'availability_365': 3}

In [72]:
# changing the leftover value (minus currency sign and commas) to a number
listing_host_df["price"] = pd.to_numeric(listing_host_df["price"])
listing_host_df.dtypes

id                                         int64
name                                      object
host_id                                    int64
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

In [32]:
# # Create new data with select columns for the AirBnB hosts
# host_df = listing_host_df[["host_id","host_name","host_since","host_location","host_response_time","host_response_rate","host_acceptance_rate","host_is_superhost","host_neighbourhood","host_listings_count","host_has_profile_pic","host_identity_verified"]].copy()

In [73]:
listing_host_df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [74]:
# delete duplicates
listing_host_df.drop_duplicates(keep="first",inplace=True)
listing_host_df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [75]:
# Store CSV to DataFrame
availability_file = "Resources/calendar.csv"
availability_df = pd.read_csv(availability_file,encoding="utf8")

In [76]:
# change column "date" to something that SQL doesn't already recognize
availability_df.rename(index=str,columns={"date":"available_date"},inplace=True)
# convert column to datetime
availability_df["available_date"]=pd.to_datetime(availability_df["available_date"])
availability_df.head()

,listing_id,available_date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [77]:
# convert boolean columns to boolean
availability_df["available"].replace(["t","f"], [True,False], inplace=True)
availability_df.head()

,listing_id,available_date,available,price
0,241032,2016-01-04,True,$85.00
1,241032,2016-01-05,True,$85.00
2,241032,2016-01-06,False,NaN
3,241032,2016-01-07,False,NaN
4,241032,2016-01-08,False,NaN


In [78]:
# remove dollar sign
availability_df["price"] = availability_df["price"].replace({'\$': '', ',': ''}, regex=True)
# convert the price column to numeric values instead of strings
availability_df["price"] = pd.to_numeric(availability_df["price"])
availability_df.head()


,listing_id,available_date,available,price
0,241032,2016-01-04,True,85.0
1,241032,2016-01-05,True,85.0
2,241032,2016-01-06,False,NaN
3,241032,2016-01-07,False,NaN
4,241032,2016-01-08,False,NaN


In [79]:
availability_df.reset_index(inplace=True)
availability_df.head()

,index,listing_id,available_date,available,price
0,0,241032,2016-01-04,True,85.0
1,1,241032,2016-01-05,True,85.0
2,2,241032,2016-01-06,False,NaN
3,3,241032,2016-01-07,False,NaN
4,4,241032,2016-01-08,False,NaN


In [80]:
#availability_df.dropna(subset=["listing_id"],inplace=True)

#availability_df[availability_df["listing_id"].isna()==True]

availability_df.rename(index=str,columns={"index":"id"},inplace=True)
availability_df.head()

,id,listing_id,available_date,available,price
0,0,241032,2016-01-04,True,85.0
1,1,241032,2016-01-05,True,85.0
2,2,241032,2016-01-06,False,NaN
3,3,241032,2016-01-07,False,NaN
4,4,241032,2016-01-08,False,NaN


In [81]:
rds_connection_string = "postgres:postgres@localhost:5432/Airbnb"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [82]:
engine.table_names()

['listings', 'property_availability']

In [83]:
listing_host_df.to_sql(name='listings', con=engine, if_exists='append', index=False)

In [84]:
availability_df.to_sql(name='property_availability', con=engine, if_exists='append', index=False)

In [85]:
pd.read_sql("select * from listings",engine).head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10,9,2018-11-19,0.10,1,0


In [86]:
pd.read_sql("select * from property_availability",engine).head()

,id,listing_id,available_date,available,price
0,0,241032,2016-01-04,True,85.0
1,1,241032,2016-01-05,True,85.0
2,2,241032,2016-01-06,False,NaN
3,3,241032,2016-01-07,False,NaN
4,4,241032,2016-01-08,False,NaN
